**Reference :** https://www.kaggle.com/khoongweihao/insect-augmentation-et-al

In [ ]:
#!pip install imutils
import sys
sys.path.append('../input/imutils/imutils-0.5.3')

* torchvision.transforms : https://dororongju.tistory.com/144

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

import torch
import torchvision
from torchvision import transforms
import tensorflow as tf
import albumentations as A
import imgaug.augmenters as iaa
from imgaug import parameters as iap

In [ ]:
DIR = "../input/plant-pathology-2021-fgvc8/train_images"
image_path = f'{DIR}/800113bb65efe69e.jpg'

In [ ]:
chosen_image = cv2.imread(image_path)
plt.imshow(chosen_image)

# **Albumentations Augmentations**

In [ ]:
albumentation_list = [A.RandomSunFlare(p=1), 
                      A.RandomFog(p=1), 
                      A.RandomBrightness(p=1),
                      A.RandomCrop(p=1,height = 512, width = 512), 
                      A.Rotate(p=1, limit=90),
                      A.RGBShift(p=1), 
                      A.RandomSnow(p=1),
                      A.HorizontalFlip(p=1), 
                      A.VerticalFlip(p=1), 
                      A.RandomContrast(limit = 0.5,p = 1),
                      A.HueSaturationValue(p=1,hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=50),
                      A.Cutout(p=1),
                      A.Transpose(p=1), 
                      A.JpegCompression(p=1),
                      A.CoarseDropout(p=1),
                      A.IAAAdditiveGaussianNoise(loc=0, scale=(2.5500000000000003, 12.75), per_channel=False, p=1),
                      A.IAAAffine(scale=1.0, translate_percent=None, translate_px=None, rotate=0.0, shear=0.0, order=1, cval=0, mode='reflect', p=1),
                      A.IAAAffine(rotate=90., p=1),
                      A.IAAAffine(rotate=180., p=1)]

In [ ]:
img_matrix_list = []
bboxes_list = []
for aug_type in albumentation_list:
    img = aug_type(image = chosen_image)['image']
    img_matrix_list.append(img)

img_matrix_list.insert(0,chosen_image)    

titles_list = ["Original","RandomSunFlare","RandomFog","RandomBrightness",
               "RandomCrop","Rotate", "RGBShift", "RandomSnow","HorizontalFlip", "VerticalFlip", "RandomContrast","HSV",
               "Cutout","Transpose","JpegCompression","CoarseDropout","IAAAdditiveGaussianNoise","IAAAffine","IAAAffineRotate90","IAAAffineRotate180"]

def plot_multiple_img(img_matrix_list, title_list, ncols, nrows=5,  main_title=""):
    fig, myaxes = plt.subplots(figsize=(20, 15), nrows=nrows, ncols=ncols, squeeze=False)
    fig.suptitle(main_title, fontsize = 30)
    fig.subplots_adjust(wspace=0.3)
    fig.subplots_adjust(hspace=0.3)
    for i, (img, title) in enumerate(zip(img_matrix_list, title_list)):
        myaxes[i // ncols][i % ncols].imshow(img)
        myaxes[i // ncols][i % ncols].set_title(title, fontsize=15)
    plt.show()
    
plot_multiple_img(img_matrix_list, titles_list, ncols = 4,main_title="Different Types of Augmentations with Albumentations")

# **imgaug Based Augmentations**

In [ ]:
ia_trans_list = [iaa.blend.BlendAlpha(factor=(0.2, 0.8),
                                      foreground=iaa.Affine(rotate=(-30, 30)),
                                      per_channel=True),
                 iaa.Fliplr(1.),
                 iaa.Flipud(1.),
                 iaa.SimplexNoiseAlpha(iaa.Multiply(iap.Choice([0.5, 1.5]), per_channel=True)),
                 iaa.Crop(percent=(0., 0.3)),
                ]

In [ ]:
img_matrix_list = []
bboxes_list = []
for aug_type in ia_trans_list:
    # convert to tensor
    chosen_image = cv2.imread(image_path)
    iaa_seq = iaa.Sequential([aug_type])
    trans_img = iaa_seq.augment_images(chosen_image)
    img_matrix_list.append(trans_img)

img_matrix_list.insert(0, chosen_image)    

titles_list = ["Original","Ghost Aug","Flip Left Right","Flip Up Down","SimplexNoiseAlpha", "Crop"]

plot_multiple_img(img_matrix_list, titles_list, ncols = 3, nrows=2, main_title="Different Types of Augmentations with Albumentations")

# **PyTorch-Based Augmentations (Torchvision)**

In [ ]:
torch_trans_list = [transforms.CenterCrop((178, 178)),
                    transforms.Resize(128),
                    transforms.RandomRotation(45),
                    transforms.RandomAffine(35),
                    transforms.RandomCrop(128),
                    transforms.RandomHorizontalFlip(p=1),
                    transforms.RandomPerspective(p=1),
                    transforms.RandomVerticalFlip(p=1)]

In [ ]:
img_matrix_list = []
bboxes_list = []
for aug_type in torch_trans_list:
    # convert to tensor
    chosen_image = cv2.imread(image_path)
    chosen_tensor = transforms.Compose([transforms.ToTensor()])(chosen_image)
    chosen_tensor = transforms.Compose([aug_type])(chosen_tensor)
    trans_img = transforms.ToPILImage()(chosen_tensor)
    img_matrix_list.append(trans_img)

img_matrix_list.insert(0, chosen_image)    

titles_list = ["Original","CenterCrop","Resize","RandomRotation","RandomAffine","RandomCrop","RandomHorizontalFlip","RandomPerspective",
               "RandomVerticalFlip"]

plot_multiple_img(img_matrix_list, titles_list, ncols = 3, nrows=3, main_title="Different Types of Augmentations with Albumentations")

# **TensorFlow-Based Augmentations**

In [ ]:
chosen_image = cv2.imread(image_path)

tf_trans_list = [
    tf.image.rot90(chosen_image, k=1), # 90 degrees counter-clockwise
    tf.image.rot90(chosen_image, k=2), # 180 degrees counter-clockwise
    tf.image.rot90(chosen_image, k=3), # 270 degrees counter-clockwise
    tf.image.random_brightness(chosen_image, 0.5), 
    tf.image.random_contrast(chosen_image, 0.2, 0.5), 
    tf.image.random_flip_left_right(chosen_image, seed=42),
    tf.image.random_flip_up_down(chosen_image, seed=42),
    tf.image.random_hue(chosen_image, 0.5),
    tf.image.random_jpeg_quality(chosen_image, 35, 50), 
    tf.image.random_saturation(chosen_image, 5, 10), 
    tf.image.transpose(chosen_image),
]

In [ ]:
img_matrix_list = []
bboxes_list = []
for aug_image in tf_trans_list:
    img_matrix_list.append(aug_image)

img_matrix_list.insert(0, chosen_image)    

titles_list = ["Original","Rotate90","Rotate180","Rotate270","RandomBrightness","RandomContrast","RandomLeftRightFlip","RandomUpDownFlip",
               "RandomHue","RandomJPEGQuality","RandomSaturation","Transpose"]

plot_multiple_img(img_matrix_list, titles_list, ncols = 3, nrows=4, main_title="Different Types of Augmentations with Albumentations")

# **Mixup**

In [ ]:
import sys
sys.path.insert(0, "../input/timm-efficientdet-pytorch")
sys.path.insert(0, "../input/omegaconf")
sys.path.insert(0, "../input/weightedboxesfusion")
import random
from glob import glob
from torch.utils.data import Dataset,DataLoader
from albumentations.pytorch.transforms import ToTensorV2
import cv2
import gc
#from tqdm import df
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedKFold

In [ ]:
marking = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')

In [ ]:
marking.tail()

In [ ]:
TRAIN_ROOT_PATH = '../input/plant-pathology-2021-fgvc8/train_images'

def collate_fn(batch):
    return tuple(zip(*batch))


class DatasetRetriever(Dataset):

    def __init__(self, marking, image_ids, transforms=None, test=False):
        super().__init__()

        self.image_ids = image_ids
        self.marking = marking
        self.transforms = transforms
        self.test = test

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]

        image, boxes = self.load_imagees(index)
        
        target = {}
        target['image_id'] = torch.tensor([index])

        if self.transforms:
            for i in range(10):
                sample = self.transforms(**{
                    'mageage': image,
                })

        return image

    def __len__(self) -> int:
        return self.image_ids.shape[0]

    def load_images(self, image_id):
        #image_id = self.image_ids[index]
        images = cv2.imread(f'{TRAIN_ROOT_PATH}/{image_id}', cv2.IMREAD_COLOR)
        #images = cv2.resize(images, dsize=(640, 480), interpolation=cv2.INTER_AREA)
        images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB).astype(np.float32)
        images /= 255
        records = self.marking[self.marking['image'] == image_id]
        return images

In [ ]:
def draw_images(list_images):
    fig, ax = plt.subplots(4, 2, figsize=(16, 32))
    for i, image in enumerate(list_images):
        ax.set_axis_off()
        ax.imshow(image);

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

df_folds = marking[['image']].copy()
f_folds = df_folds.groupby('image').count()

In [ ]:
def get_valid_transforms():
    return A.Compose(
        [
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

In [ ]:
dataset = DatasetRetriever(
    image_ids=df_folds['image'].index.values,
    marking=marking,
    transforms=get_valid_transforms(),
    test=True,
)

In [ ]:
count = 4

fig, ax = plt.subplots(count, 3, figsize=(16, 6*count))

for i in range(count):
    image = dataset.load_images(marking.loc[random.randint(0, dataset.image_ids.shape[0] - 1)]['image'])
    r_image = dataset.load_images(marking.loc[random.randint(0, dataset.image_ids.shape[0] - 1)]['image'])
    mixup_image = (image+r_image)/2
        
    ax[i][0].imshow(image)
    ax[i][1].imshow(r_image)
    ax[i][2].imshow(mixup_image)

In [ ]:
count = 4

fig, ax = plt.subplots(count, 3, figsize=(16, 6*count))

for i in range(count):
    image = dataset.load_images(marking.loc[random.randint(0, dataset.image_ids.shape[0] - 1)]['image'])
    r_image = dataset.load_images(marking.loc[random.randint(0, dataset.image_ids.shape[0] - 1)]['image'])
    
    mixup_image = image.copy()

    imsize = image.shape[0]
    x1, y1 = [int(random.uniform(imsize * 0.0, imsize * 0.45)) for _ in range(2)]
    x2, y2 = [int(random.uniform(imsize * 0.55, imsize * 1.0)) for _ in range(2)]

    
    cv2.rectangle(r_image,(x1, y1),(x2,  y2),(0, 1, 1), 5)
    
    mixup_image[y1:y2, x1:x2] = (mixup_image[y1:y2, x1:x2] + r_image[y1:y2, x1:x2])/2
    
    cv2.rectangle(mixup_image,(x1, y1),(x2,  y2),(0, 1, 1), 5)

        
    ax[i][0].imshow(image)
    ax[i][1].imshow(r_image)
    ax[i][2].imshow(mixup_image)

In [ ]:
count = 4

fig, ax = plt.subplots(count, 3, figsize=(16, 6*count))

for i in range(count):
    image = dataset.load_images(marking.loc[random.randint(0, dataset.image_ids.shape[0] - 1)]['image'])
    r_image = dataset.load_images(marking.loc[random.randint(0, dataset.image_ids.shape[0] - 1)]['image'])
    

    imsize = image.shape[0]
    w,h = imsize, imsize
    s = imsize // 2

    xc, yc = [int(random.uniform(imsize * 0.4, imsize * 0.6)) for _ in range(2)]
    direct = random.randint(0, 3)

    result_image = image.copy()
    result_boxes = []

    if direct == 0:
        x1a, y1a, x2a, y2a = max(xc - w, 0), max(yc - h, 0), xc, yc  # xmin, ymin, xmax, ymax (large image)
        x1b, y1b, x2b, y2b = w - (x2a - x1a), h - (y2a - y1a), w, h  # xmin, ymin, xmax, ymax (small image)
    elif direct == 1:  # top right
        x1a, y1a, x2a, y2a = xc, max(yc - h, 0), min(xc + w, s * 2), yc
        x1b, y1b, x2b, y2b = 0, h - (y2a - y1a), min(w, x2a - x1a), h
    elif direct == 2:  # bottom left
        x1a, y1a, x2a, y2a = max(xc - w, 0), yc, xc, min(s * 2, yc + h)
        x1b, y1b, x2b, y2b = w - (x2a - x1a), 0, max(xc, w), min(y2a - y1a, h)
    elif direct == 3:  # bottom right
        x1a, y1a, x2a, y2a = xc, yc, min(xc + w, s * 2), min(s * 2, yc + h)
        x1b, y1b, x2b, y2b = 0, 0, min(w, x2a - x1a), min(y2a - y1a, h)

    padw = x1a - x1b
    padh = y1a - y1b


    result_image[y1a:y2a, x1a:x2a] = (result_image[y1a:y2a, x1a:x2a] + r_image[y1b:y2b, x1b:x2b]) / 2 
    
    cv2.rectangle(image,(x1a, y1a),(x2a,  y2a),(0, 1, 1), 5)
    cv2.rectangle(r_image,(x1b, y1b),(x2b,  y2b),(0, 1, 1), 5)
    cv2.rectangle(result_image,(x1a, y1a),(x2a,  y2a),(0, 1, 1), 5)
    

        
    ax[i][0].imshow(image)
    ax[i][1].imshow(r_image)
    ax[i][2].imshow(result_image)